<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Hyperparameter Tuning with Keras Tuner**


Estimated time needed: **30** minutes


In this lab, you will learn how to set up Keras Tuner and prepare the environment for hyperparameter tuning. 

## Learning objectives: 
By the end of this lab, you will: 
- Install Keras Tuner and import the necessary libraries
- Load and preprocess the MNIST data set
- Define a model-building function that uses hyperparameters to configure the model architecture
- Set up Keras Tuner to search for the best hyperparameter configuration 
- Retrieve the best hyperparameters from the search and build a model with these optimized values

## Prerequisites: 
- Basic understanding of Python programming 
- Keras and TensorFlow installed


### Exercise 1: Install the Keras Tuner 

This exercise guides you through the initial setup for using Keras Tuner. You install the library, import necessary modules, and load and preprocess the MNIST data set, which will be used for hyperparameter tuning. 
1. **Install Keras Tuner:**
    - Use pip to install Keras Tuner
2. **Import necessary libraries:**
    - Import Keras Tuner, TensorFlow, and Keras modules
3. **Load and preprocess the MNIST data set:**
    - Load the MNIST data set.
    - Normalize the data set by dividing by 255.0.


In [1]:
# !pip install tensorflow==2.16.2
# !pip install keras-tuner==1.4.7
# !pip install numpy<2.0.0

#### Explanation: 
This code installs the necessary libraries using pip

- **TensorFlow**: Ensures compatibility with the Keras Tuner.
- **Keras Tuner**: The version used in this lab.
- **Numpy**: Ensures compatibility with the other installed packages.


In [2]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

#### Explanation: 
The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.


In [3]:
# Step 2: Import necessary libraries
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import mnist
from keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only



2025-02-18 20:36:09.542181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 20:36:09.586146: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 20:36:09.599646: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 20:36:09.630970: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Explanation

This code imports the necessary libraries:

- **`keras_tuner`**: Used for hyperparameter tuning.
- **`Sequential`**: A linear stack of layers in Keras.
- **`Dense`**, **`Flatten`**: Common Keras layers.
- **`mnist`**: The MNIST dataset, a standard dataset for image classification.
- **`Adam`**: An optimizer in Keras.


In [4]:

# Step 3: Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


#### Explanation

This code loads the MNIST dataset and preprocesses it:

- **`mnist.load_data()`**: Loads the dataset, returning training and validation splits.
- **`x_train / 255.0`**: Normalizes the pixel values to be between 0 and 1.
- **`print(f'...')`**: Displays the shapes of the training and validation datasets.


### Exercise 2: Defining the model with hyperparameters 

In this exercise, you define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate. This function returns a compiled Keras model that is ready for hyperparameter tuning.

**Define a model-building function:**
- Create a function `build_model` that takes a `HyperParameters` object as input.
- Use the `HyperParameters` object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.


In [5]:
# Define a model-building function 

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


#### Explanation

This function builds and compiles a Keras model with hyperparameters:

- **`hp.Int('units', ...)`**: Defines the number of units in the Dense layer as a hyperparameter.
- **`hp.Float('learning_rate', ...)`**: Defines the learning rate as a hyperparameter.
- **`model.compile()`**: Compiles the model with the Adam optimizer and sparse categorical cross-entropy loss.


### Exercise 3: Configuring the hyperparameter search 

This exercise guides you through configuring Keras Tuner. You create a `RandomSearch` tuner, specifying the model-building function, the optimization objective, the number of trials, and the directory for storing results. The search space summary provides an overview of the hyperparameters being tuned. 

**Create a RandomSearch Tuner:**
- Use the `RandomSearch` class from Keras Tuner. 
- Specify the model-building function, optimization objective (validation accuracy), number of trials, and directory for storing results.


In [6]:
# Create a RandomSearch Tuner 

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

I0000 00:00:1739898378.422482   95739 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739898378.571538   95739 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739898378.572016   95739 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739898378.590453   95739 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739898378.590957   95739 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


#### Explanation

This code sets up a Keras Tuner `RandomSearch`:

- **`build_model`**: The model-building function.
- **`objective='val_accuracy'`**: The metric to optimize (validation accuracy).
- **`max_trials=10`**: The maximum number of different hyperparameter configurations to try.
- **`executions_per_trial=2`**: The number of times to run each configuration.
- **`directory='my_dir'`**: Directory to save the results.
- **`project_name='intro_to_kt'`**: Name of the project for organizing results.

Displays a summary of the hyperparameter search space, providing an overview of the hyperparameters being tuned.



### Exercise 4: Running the hyperparameter search 

In this exercise, you run the hyperparameter search using the `search` method of the tuner. You provide the training and validation data along with the number of epochs. After the search is complete, the results summary displays the best hyperparameter configurations found. 

**Run the search:**
- Use the `search` method of the tuner. 
- Pass in the training data, validation data, and the number of epochs


In [7]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary() 

Trial 10 Complete [00h 03m 47s]
val_accuracy: 0.9760999977588654

Best val_accuracy So Far: 0.9798500239849091
Total elapsed time: 00h 35m 43s
Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
units: 448
learning_rate: 0.0006601146406744397
Score: 0.9798500239849091

Trial 00 summary
Hyperparameters:
units: 288
learning_rate: 0.0005017603502099382
Score: 0.9790000021457672

Trial 09 summary
Hyperparameters:
units: 416
learning_rate: 0.0022556211284225536
Score: 0.9760999977588654

Trial 08 summary
Hyperparameters:
units: 416
learning_rate: 0.0036377658208855486
Score: 0.9752999842166901

Trial 05 summary
Hyperparameters:
units: 352
learning_rate: 0.0002333905935392085
Score: 0.9751999974250793

Trial 03 summary
Hyperparameters:
units: 480
learning_rate: 0.0001567123683813819
Score: 0.9740999937057495

Trial 02 summary
Hyperparameters:
units: 480
learning_rate: 0.005198673201549641
Scor

#### Explanation

This command runs the hyperparameter search:

- **`epochs=5`**: Each trial is trained for 5 epochs.
- **`validation_data=(x_val, y_val)`**: The validation data to evaluate the model's performance during the search.

After the search is complete, this command displays a summary of the best hyperparameter configurations found during the search.


## Exercise 5: Analyzing and using the best hyperparameters 

In this exercise, you retrieve the best hyperparameters found during the search and print their values. You then build a model with these optimized hyperparameters and train it on the full training data set. Finally, you evaluate the model’s performance on the test set to ensure that it performs well with the selected hyperparameters. 

**Retrieve the best hyperparameters:**
- Use the `get_best_hyperparameters` method to get the best hyperparameters. 
- Print the optimal values for the hyperparameters. 

**Build and train the model:**
- Build a model using the best hyperparameters. 
- Train the model on the full training data set and evaluate its performance on the test set.


In [8]:
# Step 1: Retrieve the best hyperparameters 

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

# Step 2: Build and Train the Model with Best Hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 

 

The optimal number of units in the first dense layer is 448. 

The optimal learning rate for the optimizer is 0.0006601146406744397. 


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8799 - loss: 0.4272 - val_accuracy: 0.9610 - val_loss: 0.1323
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.9678 - loss: 0.1113 - val_accuracy: 0.9715 - val_loss: 0.0958
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.9799 - loss: 0.0667 - val_accuracy: 0.9751 - val_loss: 0.0822
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.9868 - loss: 0.0462 - val_accuracy: 0.9739 - val_loss: 0.0790
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.9905 - loss: 0.0318 - val_accuracy: 0.9766 - val_loss: 0.0768
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.9940 - loss: 0.0224 - val_accuracy: 0.9755 - val_loss: 0.0841
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0

#### Explanation

This code retrieves the best hyperparameters found during the search:

- **`get_best_hyperparameters(num_trials=1)`**: Gets the best hyperparameter configuration.
- **`print(f"...")`**: Prints the best hyperparameters.
- **`model.fit(...)`**: Trains the model on the full training data with a validation split of 20%.
- **`model.evaluate(...)`**: Evaluates the model on the test (validation) dataset and prints the accuracy, which gives an indication of how well the model generalizes.


## Practice exercises 

### Exercise 1: Setting Up Keras Tuner 

#### Objective: 
Learn how to set up Keras Tuner and prepare the environment for hyperparameter tuning. 

#### Instructions: 
1. Install Keras Tuner.
2. Import necessary libraries.
3. Load and preprocess the MNIST data set.


In [9]:
!pip install keras-tuner

In [10]:
# Write your code here
# Step 2: Import necessary libraries 
import keras_tuner as kt 
from keras.models import Sequential 
from keras.layers import Dense, Flatten 
from keras.datasets import mnist 
from keras.optimizers import Adam 

# Step 3: Load and preprocess the MNIST data set 
(x_train, y_train), (x_val, y_val) = mnist.load_data() 
x_train, x_val = x_train / 255.0, x_val / 255.0 

# Print the shapes of the training and validation datasets
print(f'Training data shape: {x_train.shape}') 
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


<details>
    <summary>Click here for Solution</summary>

```python
!pip install keras-tuner 

# Step 2: Import necessary libraries 
import keras_tuner as kt 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.optimizers import Adam 

# Step 3: Load and preprocess the MNIST data set 
(x_train, y_train), (x_val, y_val) = mnist.load_data() 
x_train, x_val = x_train / 255.0, x_val / 255.0 

# Print the shapes of the training and validation datasets
print(f'Training data shape: {x_train.shape}') 
print(f'Validation data shape: {x_val.shape}')
```

</details>


### Exercise 2: Defining the model with hyperparameters 

#### Objective: 
Define a model-building function that uses hyperparameters to configure the model architecture. 

#### Instructions: 
1. Define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate. 
2. Compile the model with sparse categorical cross-entropy loss and Adam optimizer. 


In [11]:
# Write your code here
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
import keras_tuner as kt

# Step 1: Define a model-building function
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

<details>
    <summary>Click here for Solution</summary>

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

# Step 1: Define a model-building function
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
```

</details>


### Exercise 3: Configuring the hyperparameter search 

#### Objective: 
Set up Keras Tuner to search for the best hyperparameter configuration. 

#### Instructions: 
1. Create a `RandomSearch` tuner using the model-building function. 
2. Specify the optimization objective, number of trials, and directory for storing results.


In [12]:
# Write your code here
import keras_tuner as kt

# Step 1: Create a RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,  # Ensure 'build_model' function is defined from previous code
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()

Reloading Tuner from my_dir/intro_to_kt/tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


<details>
    <summary>Click here for Solution</summary>

```python
import keras_tuner as kt

# Step 1: Create a RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,  # Ensure 'build_model' function is defined from previous code
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()
```

</details>


### Exercise 4: Running the hyperparameter search

#### Objective: 
Run the hyperparameter search and dispaly the summary of the results.

#### Instructions: 
1. Run the hyperparameter search using the `search` method of the tuner. 
2. Pass in the training data, validation data, and the number of epochs. 
3. Display a summary of the results. 


In [13]:
# Write your code here
# Step 1: Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
units: 448
learning_rate: 0.0006601146406744397
Score: 0.9798500239849091

Trial 00 summary
Hyperparameters:
units: 288
learning_rate: 0.0005017603502099382
Score: 0.9790000021457672

Trial 09 summary
Hyperparameters:
units: 416
learning_rate: 0.0022556211284225536
Score: 0.9760999977588654

Trial 08 summary
Hyperparameters:
units: 416
learning_rate: 0.0036377658208855486
Score: 0.9752999842166901

Trial 05 summary
Hyperparameters:
units: 352
learning_rate: 0.0002333905935392085
Score: 0.9751999974250793

Trial 03 summary
Hyperparameters:
units: 480
learning_rate: 0.0001567123683813819
Score: 0.9740999937057495

Trial 02 summary
Hyperparameters:
units: 480
learning_rate: 0.005198673201549641
Score: 0.9717499911785126

Trial 06 summary
Hyperparameters:
units: 416
learning_rate: 0.00015349183148720766
Score: 0.9702500104904175

Trial 07 s

<details>
    <summary>Click here for Solution</summary>

```python
# Step 1: Run the hyperparameter search 

tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

 # Display a summary of the results 

tuner.results_summary()
```

</details>


### Exercise 5: Analyzing and using the best hyperparameters 

#### Objective: 
Retrieve the best hyperparameters from the search and build a model with these optimized values. 

#### Instructions: 
1. Retrieve the best hyperparameters using the `get_best_hyperparameters` method. 
2. Build a model using the best hyperparameters. 
3. Train the model on the full training data set and evaluate its performance on the validation set.


In [14]:
# Write your code here
# Step 1: Retrieve the best hyperparameters 
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 
The optimal number of units in the first dense layer is {best_hps.get('units')}. 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
""") 

# Step 2: Build and train the model with best hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the validation set 
val_loss, val_acc = model.evaluate(x_val, y_val) 
print(f'Validation accuracy: {val_acc}')

 
The optimal number of units in the first dense layer is 448. 
The optimal learning rate for the optimizer is 0.0006601146406744397. 

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.8818 - loss: 0.4260 - val_accuracy: 0.9589 - val_loss: 0.1373
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.9678 - loss: 0.1083 - val_accuracy: 0.9688 - val_loss: 0.1039
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.9800 - loss: 0.0679 - val_accuracy: 0.9668 - val_loss: 0.1062
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.9877 - loss: 0.0439 - val_accuracy: 0.9768 - val_loss: 0.0797
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.9905 - loss: 0.0325 - val_accuracy: 0.9735 - val_loss: 0.0894
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.9933 - loss: 0.0232 - val_accuracy: 0.9775 - val_loss: 0.0756
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 24s 16ms/step - accuracy: 0.99

<details>
    <summary>Click here for Solution</summary>

```python
# Step 1: Retrieve the best hyperparameters 

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 

print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

 # Step 2: Build and train the model with best hyperparameters 

model = tuner.hypermodel.build(best_hps) 

model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

 # Evaluate the model on the validation set 

val_loss, val_acc = model.evaluate(x_val, y_val) 

print(f'Validation accuracy: {val_acc}') 
```

</details>


### Conclusion 

Congratulations on completing this lab! You have learned to set up Keras Tuner and prepare the environment for hyperparameter tuning. In addition, you defined a model-building function that uses hyperparameters to configure the model architecture. You configured Keras Tuner to search for the best hyperparameter configuration and learned to run the hyperparameter search and analyze the results. Finally, you retrieved the best hyperparameters and built a model with these optimized values. 


## Authors


Skillup


Copyright ©IBM Corporation. All rights reserved.
